In [1]:
import numpy as np
import pandas as pd
import dill as dl
import matplotlib.pyplot as plt
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f

In [2]:
with open("D:/DZ/Course_6/Diploma/Data/two_time_points.csv", "rb") as inp:
    two_time_points = pickle.load(inp)

In [3]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [4]:
with open("D:/DZ/Course_6/Diploma/Data/mun_data_75.csv", "rb") as inp:
    data_for_models = pd.read_csv(inp, index_col=0)

In [5]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [6]:
#with open("D:/DZ/Course_6/Diploma/Data/mun_data_25.csv", "rb") as inp:
#    mun_data = pd.read_csv(inp, index_col=0)

In [7]:
pd.set_option("max_columns", None)

In [8]:
two_time_points

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_period
0,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0
1,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,1
2,Хасанский район,Приморский край,0,1,2016,NaN,0.0,NaN,4.20,32139.0,NaN,0.023735,0.011735,NaN,0
3,Хасанский район,Приморский край,0,1,2021,NaN,NaN,NaN,6.20,30318.0,NaN,0.020031,0.004033,0.002294,1
4,Артемовский,Приморский край,1,1,2016,NaN,0.0,NaN,2.62,114873.0,NaN,0.009542,0.009249,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Мошенской район,Новгородская область,0,0,2021,45.00,NaN,NaN,3.32,5797.0,NaN,NaN,0.007950,NaN,1
304,Пестовский район,Новгородская область,0,1,2016,88.77,0.0,NaN,0.63,20934.0,NaN,NaN,0.003777,NaN,0
305,Пестовский район,Новгородская область,0,1,2021,75.10,NaN,NaN,0.82,19579.0,NaN,NaN,0.009602,0.014286,1
306,Хвойнинский район,Новгородская область,0,1,2016,27.61,10.0,NaN,24.65,14950.0,NaN,NaN,0.003137,0.012089,0


In [9]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [10]:
temp_fin['values'].min()

-46465.5756015087

In [11]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [12]:
full_ind_data = pd.merge(full_ind_data, data_for_models, on=["raion", "region"], how="left").drop(columns=["treatment_1_x", "treatment_2_x"]).rename(columns={"treatment_1_y": "treatment_1", "treatment_2_y": "treatment_2"}).reset_index(drop=True)

In [13]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

# Goods

In [14]:
full_ind_data.year.max()

2021

In [15]:
for i in range(-8,4):
    if i < 0:
        string = f"minus_{abs(i)}"
    else:
        string = f'plus_{i}'
    full_ind_data[string] = 0
    for region in mapping.keys():
        full_ind_data.loc[(full_ind_data.region == region) & (full_ind_data.year == mapping[region]+5+i) & (full_ind_data.treatment_1 == 1), string] = 1

In [16]:
full_ind_data.head(10)

,raion,region,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Владивостокский,Приморский край,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Владивостокский,Приморский край,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Владивостокский,Приморский край,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Владивостокский,Приморский край,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Владивостокский,Приморский край,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,Владивостокский,Приморский край,2018,NaN,0.0,34023.34,4.21,633102.0,NaN,0.016002,0.005096,0.002653,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,Владивостокский,Приморский край,2019,NaN,0.0,67570.22,4.48,633144.0,NaN,0.028937,0.013167,0.002075,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,Владивостокский,Приморский край,2020,NaN,NaN,106191.72,4.80,634665.0,NaN,0.025499,0.007071,0.001982,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,Владивостокский,Приморский край,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,Хасанский район,Приморский край,2013,NaN,6.3,116999.00,4.30,34112.0,NaN,0.028997,0.009722,0.008197,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
treat_1_roads = full_ind_data[["raion", "region", "bad_roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", 
                 data=treat_1_roads).fit()
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.843
Method:                 Least Squares   F-statistic:                     39.34
Date:                Fri, 12 May 2023   Prob (F-statistic):          5.41e-280
Time:                        23:04:03   Log-Likelihood:                -3514.1
No. Observations:                 955   AIC:                             7298.
Df Residuals:                     820   BIC:                             7955.
Df Model:                         134                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           11.6575      3.842      3.034      0.002       4.116      19.199
C(raion)[T.Антроповский район]                                      75.1525      5.031     14.939      0.000      65.278      85.027
C(raion)[T.Аяно-Майский район]                                      88.3425     11.040      8.002      0.000      66.673     110.012
C(raion)[T.Батецкий район]                                          51.7481      5.031     10.286      0.000      41.873      61.623
C(raion)[T.Бикинский район]                                         21.2425      5.175      4.105      0.000      11.085      31.400
C(raion)[T.Боровичский район]                                       30.3781      5.031      6.038      0.000      20.503      40.253
C(raion)[T.Буйский район]                                           30.2692      5.031      6.017      0.000      20.394      40.144
C(raion)[T.Валдайский район]                                        27.6847      5.031      5.503      0.000      17.810      37.559
C(raion)[T.Ванинский район]                                         13.0312      5.175      2.518      0.012       2.874      23.189
C(raion)[T.Великий Новгород]                                        25.8925      5.031      5.147      0.000      16.018      35.767
C(raion)[T.Верхнебуреинский район]                                   9.5336      5.907      1.614      0.107      -2.062      21.129
C(raion)[T.Волжский район]                                          68.9225      5.031     13.700      0.000      59.048      78.797
C(raion)[T.Волотовский район]                                       20.5820      5.178      3.975      0.000      10.418      30.746
C(raion)[T.Вохомский район]                                         75.3192      5.031     14.972      0.000      65.444      85.194
C(raion)[T.Вяземский район]                                         -4.4053      5.031     -0.876      0.381     -14.280       5.469
C(raion)[T.Галичский район]                                         74.1792      5.031     14.745      0.000      64.304      84.054
C(raion)[T.Горномарийский район]                                    81.1903      5.031     16.139      0.000      71.316      91.065
C(raion)[T.Город Волжск]                                            19.3914      5.031      3.855      0.000       9.517      29.266
C(raion)[T.Город Горно-Алтайск]                                     15.6825      5.031      3.117      0.002       5.808      25.557
C(raion)[T.Город Козьмодемьянск]                                    55.4780      6.348      8.739      0.000      43.018      67.938
C(raion)[T.Город Комсомо

In [18]:
treat_1_roads.head()

,raion,region,bad_roads,year,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
281,Печенгский район,Мурманская область,9.00,2015,0,0,0,0,0,0,0,0,0,0,0,0
282,Печенгский район,Мурманская область,6.26,2016,0,0,0,0,0,0,0,0,0,0,0,0
283,Печенгский район,Мурманская область,7.86,2017,0,0,0,0,0,0,0,0,0,0,0,0
284,Печенгский район,Мурманская область,11.72,2018,0,0,0,0,0,0,0,0,0,0,0,0
285,Печенгский район,Мурманская область,24.80,2019,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
treat_1_schools = full_ind_data[["raion", "region", "bad_schools", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_schools).fit()
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.439
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     3.242
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.35e-20
Time:                        23:04:05   Log-Likelihood:                -2421.0
No. Observations:                 638   AIC:                             5092.
Df Residuals:                     513   BIC:                             5649.
Df Model:                         124                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               52.2711      8.653      6.041      0.000      35.271      69.271
C(raion)[T.Анучинский район]           -51.6100     10.065     -5.128      0.000     -71.383     -31.837
C(raion)[T.Арсеньевский]               -51.6100     10.065     -5.128      0.000     -71.383     -31.837
C(raion)[T.Артемовский]                -55.4161      9.833     -5.636      0.000     -74.735     -36.098
C(raion)[T.Аяно-Майский район]         -25.2424     14.722     -1.715      0.087     -54.165       3.680
C(raion)[T.Батецкий район]             -44.9087      9.826     -4.570      0.000     -64.213     -25.605
C(raion)[T.Бикинский район]            -41.4511     12.039     -3.443      0.001     -65.103     -17.799
C(raion)[T.Боровичский район]          -51.5754      9.826     -5.249      0.000     -70.879     -32.272
C(raion)[T.Валдайский район]           -51.5754      9.826     -5.249      0.000     -70.879     -32.272
C(raion)[T.Ванинский район]            -44.9597     14.800     -3.038      0.003     -74.036     -15.883
C(raion)[T.Великий Новгород]           -47.0052      9.421     -4.989      0.000     -65.515     -28.496
C(raion)[T.Владивостокский]            -51.6100     10.065     -5.128      0.000     -71.383     -31.837
C(raion)[T.Волжский район]             -51.9830      9.421     -5.517      0.000     -70.492     -33.474
C(raion)[T.Волотовский район]          -34.9087      9.826     -3.553      0.000     -54.213     -15.605
C(raion)[T.Горномарийский район]       -50.0115      9.651     -5.182      0.000     -68.973     -31.050
C(raion)[T.Город Волжск]               -51.7909      9.523     -5.439      0.000     -70.500     -33.082
C(raion)[T.Город Горно-Алтайск]        -50.4830      9.421     -5.358      0.000     -68.992     -31.974
C(raion)[T.Город Йошкар-Ола]           -48.8534      9.523     -5.130      0.000     -67.562     -30.145
C(raion)[T.Город Козьмодемьянск]       -51.5972      9.651     -5.346      0.000     -70.559     -32.636
C(raion)[T.Город Новомосковск]         -55.0676      9.834     -5.600      0.000     -74.388     -35.748
C(raion)[T.Город Тула]                 -50.0718     10.066     -4.974      0.000     -69.847     -30.296
C(raion)[T.Дальнегорский]              -48.9300     10.065     -4.862      0.000     -68.703     -29.157
C(raion)[T.Дальнереченский]            -48.7500     10.065     -4.844      0.000     -68.523     -28.977
C(raion)[T.Демянский район]            -39.0754      9.826     -3.977      0.000     -58.379     -19.772
C(raion)[T.ЗАТО Фокино]                -52.6833     11.018     -4.781      0.000     -74.330     -31.037
C(raion)[T.Заокский район]             -16.4118     10.066     -1.630      0.104     -36.187       3.364
C(raion)[T.Звениговский район]        

In [20]:
treat_1_sewerage =  full_ind_data[["raion", "region", "year", "sewage_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_sewerage).fit()
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.636
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     5.575
Date:                Fri, 12 May 2023   Prob (F-statistic):           3.40e-37
Time:                        23:04:05   Log-Likelihood:                 1044.6
No. Observations:                 491   AIC:                            -1853.
Df Residuals:                     373   BIC:                            -1358.
Df Model:                         117                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0074      0.015      0.483      0.630      -0.023       0.038
C(raion)[T.Анучинский район]                                        -0.0066      0.037     -0.179      0.858      -0.078       0.065
C(raion)[T.Арсеньевский]                                            -0.0088      0.024     -0.362      0.717      -0.057       0.039
C(raion)[T.Артемовский]                                             -0.0054      0.019     -0.279      0.780      -0.044       0.033
C(raion)[T.Бикинский район]                                          0.0018      0.028      0.065      0.948      -0.053       0.057
C(raion)[T.Боровичский район]                                       -0.0034      0.019     -0.173      0.863      -0.042       0.035
C(raion)[T.Буйский район]                                           -0.0057      0.028     -0.203      0.839      -0.060       0.049
C(raion)[T.Валдайский район]                                         0.0046      0.019      0.250      0.803      -0.032       0.041
C(raion)[T.Ванинский район]                                         -0.0062      0.019     -0.320      0.749      -0.044       0.032
C(raion)[T.Великий Новгород]                                        -0.0016      0.019     -0.083      0.934      -0.040       0.037
C(raion)[T.Верхнебуреинский район]                                   0.0074      0.028      0.265      0.791      -0.047       0.062
C(raion)[T.Владивостокский]                                         -0.0054      0.019     -0.293      0.770      -0.042       0.031
C(raion)[T.Вяземский район]                                         -0.0063      0.022     -0.281      0.779      -0.050       0.038
C(raion)[T.Галичский район]                                          0.0370      0.036      1.014      0.311      -0.035       0.109
C(raion)[T.Город Горно-Алтайск]                                      0.0026      0.037      0.070      0.944      -0.069       0.074
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0051      0.019     -0.260      0.795      -0.043       0.033
C(raion)[T.Город Новомосковск]                                      -0.0051      0.028     -0.183      0.855      -0.060       0.050
C(raion)[T.Город Тула]                                              -0.0046      0.022     -0.207      0.836      -0.048       0.039
C(raion)[T.Город Хабаровск]                                          0.0016      0.019      0.086      0.932      -0.035       0.038
C(raion)[T.Дальнегорский]                                           -0.0084      0.019     -0.431      0.666      -0.047       0.030
C(raion)[T.Дальнереченск

In [21]:
treat_1_heat = full_ind_data[["raion", "region", "year", "heat_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_heat).fit() 
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     2.076
Date:                Fri, 12 May 2023   Prob (F-statistic):           0.000269
Time:                        23:04:07   Log-Likelihood:                 627.07
No. Observations:                 260   AIC:                            -1160.
Df Residuals:                     213   BIC:                            -992.8
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0279      0.011      2.433      0.016       0.005       0.050
C(raion)[T.Арсеньевский]             -0.0008      0.013     -0.058      0.953      -0.027       0.026
C(raion)[T.Артемовский]              -0.0086      0.013     -0.638      0.524      -0.035       0.018
C(raion)[T.Владивостокский]           0.0039      0.013      0.289      0.773      -0.023       0.030
C(raion)[T.Дальнегорский]            -0.0068      0.013     -0.508      0.612      -0.033       0.020
C(raion)[T.Дальнереченский]           0.0094      0.013      0.699      0.485      -0.017       0.036
C(raion)[T.ЗАТО Фокино]               0.0549      0.015      3.765      0.000       0.026       0.084
C(raion)[T.Кавалеровский район]       0.0014      0.021      0.065      0.948      -0.040       0.043
C(raion)[T.Кировский район]           0.0094      0.013      0.699      0.486      -0.017       0.036
C(raion)[T.Красноармейский район]     0.0284      0.021      1.341      0.181      -0.013       0.070
C(raion)[T.Лазовский район]           0.0379      0.014      2.758      0.006       0.011       0.065
C(raion)[T.Лесозаводский]             0.0152      0.013      1.129      0.260      -0.011       0.042
C(raion)[T.Михайловский район]        0.0105      0.013      0.785      0.434      -0.016       0.037
C(raion)[T.Надеждинский район]        0.0260      0.013      1.936      0.054      -0.000       0.053
C(raion)[T.Находкинский]              0.0097      0.013      0.724      0.470      -0.017       0.036
C(raion)[T.Октябрьский район]         0.0138      0.014      1.006      0.316      -0.013       0.041
C(raion)[T.Ольгинский район]          0.0053      0.013      0.393      0.695      -0.021       0.032
C(raion)[T.Партизанский]              0.0019      0.013      0.142      0.888      -0.025       0.028
C(raion)[T.Пограничный район]         0.0343      0.015      2.359      0.019       0.006       0.063
C(raion)[T.Пожарский район]          -0.0101      0.013     -0.748      0.455      -0.037       0.016
C(raion)[T.Спасск-Дальний]            0.0023      0.013      0.168      0.867      -0.024       0.029
C(raion)[T.Спасский район]           -0.0019      0.013     -0.138      0.890      -0.028       0.025
C(raion)[T.Тернейский район]         -0.0067      0.014     -0.485      0.628      -0.034       0.020
C(raion)[T.Уссурийский]               0.0177      0.013      1.317      0.189      -0.009       0.044
C(raion)[T.Ханкайский район]       6.047e-05      0.014      0.004      0.996      -0.027       0.027
C(raion)[T.Хасанский район]           0.0113      0.013      0.840      0.402      -0.015       0.038
C(raion)[T.Хорольский район]          0.0178      0.014      1.295      0.197      -0.009       0.045
C(raion)[T.Черниговский

In [22]:
treat_1_water = full_ind_data[["raion", "region", "year", "water_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_water).fit()
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.226
Method:                 Least Squares   F-statistic:                     2.725
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.57e-16
Time:                        23:04:08   Log-Likelihood:                 1545.0
No. Observations:                 756   AIC:                            -2832.
Df Residuals:                     627   BIC:                            -2235.
Df Model:                         128                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0426      0.012      3.537      0.000       0.019       0.066
C(raion)[T.Антроповский район]                                      -0.0311      0.023     -1.352      0.177      -0.076       0.014
C(raion)[T.Анучинский район]                                        -0.0175      0.018     -0.965      0.335      -0.053       0.018
C(raion)[T.Арсеньевский]                                            -0.0309      0.023     -1.343      0.180      -0.076       0.014
C(raion)[T.Артемовский]                                             -0.0161      0.016     -0.991      0.322      -0.048       0.016
C(raion)[T.Батецкий район]                                          -0.0307      0.019     -1.594      0.111      -0.068       0.007
C(raion)[T.Бикинский район]                                         -0.0027      0.021     -0.130      0.897      -0.043       0.038
C(raion)[T.Боровичский район]                                       -0.0264      0.016     -1.624      0.105      -0.058       0.006
C(raion)[T.Буйский район]                                           -0.0286      0.018     -1.573      0.116      -0.064       0.007
C(raion)[T.Валдайский район]                                        -0.0195      0.016     -1.204      0.229      -0.051       0.012
C(raion)[T.Ванинский район]                                         -0.0282      0.016     -1.738      0.083      -0.060       0.004
C(raion)[T.Великий Новгород]                                        -0.0330      0.017     -1.970      0.049      -0.066      -0.000
C(raion)[T.Верхнебуреинский район]                                  -0.0140      0.019     -0.727      0.467      -0.052       0.024
C(raion)[T.Владивостокский]                                         -0.0288      0.016     -1.777      0.076      -0.061       0.003
C(raion)[T.Волотовский район]                                       -0.0085      0.019     -0.442      0.659      -0.046       0.029
C(raion)[T.Вохомский район]                                         -0.0216      0.019     -1.123      0.262      -0.059       0.016
C(raion)[T.Вяземский район]                                         -0.0211      0.017     -1.263      0.207      -0.054       0.012
C(raion)[T.Галичский район]                                         -0.0345      0.019     -1.795      0.073      -0.072       0.003
C(raion)[T.Город Горно-Алтайск]                                     -0.0283      0.016     -1.743      0.082      -0.060       0.004
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0155      0.016     -0.955      0.340      -0.047       0.016
C(raion)[T.Город Хабаров

In [23]:
table = pd.DataFrame(np.array([roads.params[(len(roads.params)-10):(len(roads.params))].values, 
                               schools.params[(len(schools.params)-10):(len(schools.params))].values, 
                               sewerage.params[(len(sewerage.params)-10):(len(sewerage.params))].values,
                               heat.params[(len(heat.params)-10):(len(heat.params))].values,
                               water.params[(len(water.params)-10):(len(water.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Качество дорог", "Состояние дошкольных учреждений", "Объем ремонта канализационной сети", "Объем ремонта тепловой сети", "Объем ремонта водопроводной сети"])

In [24]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная
0,-1.359793e-14,1.182975e+01,4.542178,4.484662,-3.278415,0.622106,0.090730,0.494125,0.562312,2.785253,Качество дорог
1,-1.831361e-15,6.846854e-15,-8.760415,-4.042345,-7.003226,-6.361899,-9.398117,-1.582746,0.000000,0.000000,Состояние дошкольных учреждений
2,3.068148e-17,3.956186e-02,0.033431,0.057009,0.054755,0.041293,0.034776,0.039185,-0.006462,0.029679,Объем ремонта канализационной сети
3,-2.713358e-02,-2.340395e-03,0.006403,0.002837,-0.004186,0.016120,-0.016953,-0.002834,0.000000,0.000000,Объем ремонта тепловой сети
4,1.553369e-02,1.147049e-02,0.019118,0.007609,0.006739,0.004838,0.010219,0.005567,0.006476,0.000000,Объем ремонта водопроводной сети


In [25]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_75_cor.xlsx")

# Models for financial data

In [26]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [27]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [28]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [29]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [30]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [31]:
frames = [actual, initial, difference]

In [32]:
for j in range(len(frames)):
    #temp_fin_data = pd.DataFrame()
    #for j in mapping.keys():
    #    temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 5))])
    frames[j]["treatment_period"] = [1 if i > 2018 else 0 for i in  frames[j].year]
    frames[j] = pd.merge(frames[j], data_for_models, on=["region", "raion"], how="inner").reset_index(drop=True)

In [33]:
for j in range(len(frames)): 
    for i in range(-8,4):
        if i < 0:
            string = f"minus_{abs(i)}"
        else:
            string = f'plus_{i}'
        frames[j][string] = 0
        for region in mapping.keys():
            frames[j].loc[(frames[j].region == region) & (frames[j].year == mapping[region]+5+i) & (frames[j].treatment_1 == 1), string] = 1

In [34]:
frames[0].loc[frames[0].treatment_1 == 1]

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
90,Костромская область,Межевской район,2013,119.932022,447.681476,13080.116533,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
91,Костромская область,Межевской район,2014,268.793039,305.110363,14721.070617,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
92,Костромская область,Межевской район,2015,366.898199,960.397961,21905.499499,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
93,Костромская область,Межевской район,2016,692.608602,752.513135,12733.019118,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
94,Костромская область,Межевской район,2017,NaN,NaN,NaN,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1335,Тульская область,Ясногорский район,2018,3154.085535,875.742065,11384.009946,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1336,Тульская область,Ясногорский район,2019,2405.870106,745.600890,11821.274934,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1337,Тульская область,Ясногорский район,2020,NaN,NaN,NaN,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1338,Тульская область,Ясногорский район,2021,NaN,4616.592141,13202.097917,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [35]:
frames[2].year.max()

2022

In [36]:
#frames[0]

In [37]:
actual, initial, difference = frames

In [38]:
actual.head()

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
actual_roads_data = actual[["raion", "region", "roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_roads_data).fit()
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.467
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     5.130
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.02e-60
Time:                        23:04:22   Log-Likelihood:                -9484.2
No. Observations:                1147   AIC:                         1.930e+04
Df Residuals:                     979   BIC:                         2.015e+04
Df Model:                         167                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          305.0003    351.898      0.867      0.386    -385.561     995.562
C(raion)[T.Антроповский район]                                     560.7618    570.433      0.983      0.326    -558.651    1680.175
C(raion)[T.Анучинский район]                                       778.3277    515.049      1.511      0.131    -232.400    1789.055
C(raion)[T.Арсеньевский]                                           512.3630    481.504      1.064      0.288    -432.535    1457.261
C(raion)[T.Артемовский]                                            981.1595    481.504      2.038      0.042      36.262    1926.057
C(raion)[T.Аяно-Майский район]                                    2560.3153    481.504      5.317      0.000    1615.418    3505.213
C(raion)[T.Батецкий район]                                        -247.2697    481.504     -0.514      0.608   -1192.167     697.628
C(raion)[T.Бикинский район]                                       -320.8584    481.504     -0.666      0.505   -1265.756     624.039
C(raion)[T.Боровичский район]                                       59.6728    481.504      0.124      0.901    -885.225    1004.571
C(raion)[T.Буйский район]                                          883.3736    570.433      1.549      0.122    -236.039    2002.786
C(raion)[T.Валдайский район]                                       169.6356    481.504      0.352      0.725    -775.262    1114.533
C(raion)[T.Ванинский район]                                        -81.9262    481.504     -0.170      0.865   -1026.824     862.971
C(raion)[T.Великий Новгород]                                      1361.8925    481.504      2.828      0.005     416.995    2306.790
C(raion)[T.Верхнебуреинский район]                                 732.7835    481.504      1.522      0.128    -212.114    1677.681
C(raion)[T.Владивостокский]                                       2130.3599    481.504      4.424      0.000    1185.462    3075.258
C(raion)[T.Волжский район]                                          39.8435    496.472      0.080      0.936    -934.428    1014.115
C(raion)[T.Волотовский район]                                     -155.3736    496.472     -0.313      0.754   -1129.645     818.898
C(raion)[T.Вохомский район]                                       2149.2144    570.435      3.768      0.000    1029.798    3268.631
C(raion)[T.Вяземский район]                                        121.2266    481.504      0.252      0.801    -823.671    1066.124
C(raion)[T.Галичский район]                                        850.0162    538.806      1.578      0.115    -207.331    1907.363
C(raion)[T.Горномарийски

In [40]:
actual_housing_data = actual[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_housing_data).fit()
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     45.05
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:04:22   Log-Likelihood:                -11274.
No. Observations:                1163   AIC:                         2.289e+04
Df Residuals:                     994   BIC:                         2.374e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         2273.0515   1458.872      1.558      0.120    -589.772    5135.875
C(raion)[T.Антроповский район]                                    -384.7812   2370.004     -0.162      0.871   -5035.566    4266.004
C(raion)[T.Анучинский район]                                     -1143.2535   2139.928     -0.534      0.593   -5342.549    3056.042
C(raion)[T.Арсеньевский]                                           799.8042   2000.568      0.400      0.689   -3126.017    4725.625
C(raion)[T.Артемовский]                                            856.0682   2000.568      0.428      0.669   -3069.753    4781.889
C(raion)[T.Аяно-Майский район]                                    5.699e+04   2000.568     28.488      0.000    5.31e+04    6.09e+04
C(raion)[T.Батецкий район]                                        -442.0961   2000.568     -0.221      0.825   -4367.917    3483.725
C(raion)[T.Бикинский район]                                       1084.5129   2000.568      0.542      0.588   -2841.308    5010.334
C(raion)[T.Боровичский район]                                     -488.6702   2000.568     -0.244      0.807   -4414.491    3437.151
C(raion)[T.Буйский район]                                        -1376.7128   2554.354     -0.539      0.590   -6389.259    3635.833
C(raion)[T.Валдайский район]                                      -750.3068   2000.568     -0.375      0.708   -4676.128    3175.514
C(raion)[T.Ванинский район]                                       9769.9337   2000.568      4.884      0.000    5844.113    1.37e+04
C(raion)[T.Великий Новгород]                                       921.6896   2000.568      0.461      0.645   -3004.132    4847.511
C(raion)[T.Верхнебуреинский район]                                5032.4429   2000.568      2.516      0.012    1106.622    8958.264
C(raion)[T.Владивостокский]                                        752.1343   2000.568      0.376      0.707   -3173.687    4677.955
C(raion)[T.Волжский район]                                        -961.1513   2062.753     -0.466      0.641   -5009.001    3086.698
C(raion)[T.Волотовский район]                                      848.8349   2062.753      0.412      0.681   -3199.015    4896.684
C(raion)[T.Вохомский район]                                        360.7566   2370.001      0.152      0.879   -4290.024    5011.537
C(raion)[T.Вяземский район]                                       2856.1783   2000.568      1.428      0.154   -1069.643    6781.999
C(raion)[T.Галичский район]                                      -1163.4012   2238.619     -0.520      0.603   -5556.362    3229.560
C(raion)[T.Горномарийски

In [41]:
actual_education_data = actual[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_education_data).fit()
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     53.20
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:04:23   Log-Likelihood:                -11055.
No. Observations:                1182   AIC:                         2.245e+04
Df Residuals:                    1013   BIC:                         2.331e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.759e+04   1034.921     16.993      0.000    1.56e+04    1.96e+04
C(raion)[T.Антроповский район]                                   -1866.8504   1682.746     -1.109      0.268   -5168.916    1435.216
C(raion)[T.Анучинский район]                                     -3831.0642   1519.412     -2.521      0.012   -6812.620    -849.509
C(raion)[T.Арсеньевский]                                         -6579.0110   1420.462     -4.632      0.000   -9366.396   -3791.626
C(raion)[T.Артемовский]                                          -6738.1640   1420.462     -4.744      0.000   -9525.549   -3950.779
C(raion)[T.Аяно-Майский район]                                    4.413e+04   1420.462     31.068      0.000    4.13e+04    4.69e+04
C(raion)[T.Батецкий район]                                       -4923.0793   1420.462     -3.466      0.001   -7710.464   -2135.695
C(raion)[T.Бикинский район]                                       -657.5799   1420.462     -0.463      0.644   -3444.964    2129.805
C(raion)[T.Боровичский район]                                    -7591.8127   1420.462     -5.345      0.000   -1.04e+04   -4804.428
C(raion)[T.Буйский район]                                        -3098.4174   1682.746     -1.841      0.066   -6400.483     203.649
C(raion)[T.Валдайский район]                                     -7065.9048   1420.462     -4.974      0.000   -9853.289   -4278.520
C(raion)[T.Ванинский район]                                        504.7007   1420.462      0.355      0.722   -2282.684    3292.085
C(raion)[T.Великий Новгород]                                     -8048.9226   1420.462     -5.666      0.000   -1.08e+04   -5261.538
C(raion)[T.Верхнебуреинский район]                                6874.1566   1420.462      4.839      0.000    4086.772    9661.541
C(raion)[T.Владивостокский]                                      -8211.9027   1420.462     -5.781      0.000    -1.1e+04   -5424.518
C(raion)[T.Волжский район]                                       -6136.8002   1464.619     -4.190      0.000   -9010.834   -3262.767
C(raion)[T.Волотовский район]                                    -7120.0528   1464.619     -4.861      0.000   -9994.086   -4246.019
C(raion)[T.Вохомский район]                                        401.3817   1682.752      0.239      0.812   -2900.698    3703.461
C(raion)[T.Вяземский район]                                       3552.5245   1420.462      2.501      0.013     765.140    6339.909
C(raion)[T.Галичский район]                                      -3938.7439   1589.470     -2.478      0.013   -7057.774    -819.713
C(raion)[T.Горномарийски

In [42]:
initial_roads_data = initial[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_roads_data).fit()
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     2.372
Date:                Fri, 12 May 2023   Prob (F-statistic):           6.01e-14
Time:                        23:04:23   Log-Likelihood:                -7859.6
No. Observations:                 911   AIC:                         1.601e+04
Df Residuals:                     765   BIC:                         1.671e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          101.9540    548.730      0.186      0.853    -975.241    1179.149
C(raion)[T.Антроповский район]                                     675.1916    737.038      0.916      0.360    -771.665    2122.048
C(raion)[T.Анучинский район]                                       812.7348    841.360      0.966      0.334    -838.914    2464.383
C(raion)[T.Арсеньевский]                                           161.1737    737.038      0.219      0.827   -1285.683    1608.030
C(raion)[T.Артемовский]                                            492.8104    737.038      0.669      0.504    -954.046    1939.667
C(raion)[T.Аяно-Майский район]                                    2700.7786    737.038      3.664      0.000    1253.922    4147.635
C(raion)[T.Батецкий район]                                        -160.3501    763.301     -0.210      0.834   -1658.763    1338.063
C(raion)[T.Бикинский район]                                       -132.7269    737.038     -0.180      0.857   -1579.583    1314.129
C(raion)[T.Боровичский район]                                      169.4364    763.301      0.222      0.824   -1328.976    1667.849
C(raion)[T.Буйский район]                                          488.8160    763.301      0.640      0.522   -1009.597    1987.229
C(raion)[T.Валдайский район]                                       189.0843    763.301      0.248      0.804   -1309.328    1687.497
C(raion)[T.Ванинский район]                                        -26.5885    737.038     -0.036      0.971   -1473.445    1420.268
C(raion)[T.Великий Новгород]                                      -214.6817    737.038     -0.291      0.771   -1661.538    1232.175
C(raion)[T.Верхнебуреинский район]                                -215.3413    737.038     -0.292      0.770   -1662.198    1231.515
C(raion)[T.Владивостокский]                                       1271.0419    737.038      1.725      0.085    -175.814    2717.898
C(raion)[T.Волотовский район]                                     -171.4103    763.145     -0.225      0.822   -1669.518    1326.697
C(raion)[T.Вохомский район]                                        736.6153    763.301      0.965      0.335    -761.797    2235.028
C(raion)[T.Вяземский район]                                        109.7405    737.038      0.149      0.882   -1337.116    1556.597
C(raion)[T.Галичский район]                                        746.0257    763.301      0.977      0.329    -752.387    2244.438
C(raion)[T.Город Горно-Алтайск]                                   1266.2961    904.208      1.400      0.162    -508.727    3041.319
C(raion)[T.Город Комсомо

In [43]:
initial_housing_data = initial[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_housing_data).fit()
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.749
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     15.87
Date:                Fri, 12 May 2023   Prob (F-statistic):          2.41e-158
Time:                        23:04:24   Log-Likelihood:                -9364.3
No. Observations:                 929   AIC:                         1.902e+04
Df Residuals:                     781   BIC:                         1.974e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          799.1784   2318.222      0.345      0.730   -3751.505    5349.862
C(raion)[T.Антроповский район]                                   -1037.0589   3862.105     -0.269      0.788   -8618.394    6544.276
C(raion)[T.Анучинский район]                                      -407.3317   3593.631     -0.113      0.910   -7461.651    6646.988
C(raion)[T.Арсеньевский]                                           928.4250   3148.321      0.295      0.768   -5251.749    7108.599
C(raion)[T.Артемовский]                                            184.4478   3148.321      0.059      0.953   -5995.726    6364.621
C(raion)[T.Аяно-Майский район]                                    4.955e+04   3148.321     15.739      0.000    4.34e+04    5.57e+04
C(raion)[T.Батецкий район]                                        -917.8584   3260.131     -0.282      0.778   -7317.516    5481.799
C(raion)[T.Бикинский район]                                       1277.5350   3148.321      0.406      0.685   -4902.639    7457.709
C(raion)[T.Боровичский район]                                     -655.7229   3260.131     -0.201      0.841   -7055.380    5743.934
C(raion)[T.Буйский район]                                         -944.7772   3260.131     -0.290      0.772   -7344.435    5454.880
C(raion)[T.Валдайский район]                                      -908.6223   3403.226     -0.267      0.790   -7589.176    5771.931
C(raion)[T.Ванинский район]                                       9064.7576   3148.321      2.879      0.004    2884.584    1.52e+04
C(raion)[T.Великий Новгород]                                      -958.7096   3148.321     -0.305      0.761   -7138.883    5221.464
C(raion)[T.Верхнебуреинский район]                                -957.1933   3148.321     -0.304      0.761   -7137.367    5222.980
C(raion)[T.Владивостокский]                                        590.1534   3148.321      0.187      0.851   -5590.020    6770.327
C(raion)[T.Волотовский район]                                    -1005.1123   3259.829     -0.308      0.758   -7404.177    5393.953
C(raion)[T.Вохомский район]                                       -742.8279   3260.131     -0.228      0.820   -7142.485    5656.829
C(raion)[T.Вяземский район]                                       2499.8620   3148.321      0.794      0.427   -3680.312    8680.036
C(raion)[T.Галичский район]                                       -784.2292   3260.131     -0.241      0.810   -7183.887    5615.428
C(raion)[T.Город Горно-Алтайск]                                   1182.3370   3862.105      0.306      0.760   -6398.998    8763.672
C(raion)[T.Город Комсомо

In [44]:
initial_education_data = initial[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_education_data).fit()
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     104.1
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:04:24   Log-Likelihood:                -8532.0
No. Observations:                 953   AIC:                         1.736e+04
Df Residuals:                     805   BIC:                         1.808e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.574e+04    748.767     21.020      0.000    1.43e+04    1.72e+04
C(raion)[T.Антроповский район]                                   -6153.2268   1017.524     -6.047      0.000   -8150.540   -4155.914
C(raion)[T.Анучинский район]                                     -2249.2007   1161.418     -1.937      0.053   -4528.965      30.564
C(raion)[T.Арсеньевский]                                         -5571.2562   1017.524     -5.475      0.000   -7568.569   -3573.943
C(raion)[T.Артемовский]                                          -6552.1787   1017.524     -6.439      0.000   -8549.492   -4554.866
C(raion)[T.Аяно-Майский район]                                    3.437e+04   1017.524     33.779      0.000    3.24e+04    3.64e+04
C(raion)[T.Батецкий район]                                       -4364.4548   1053.653     -4.142      0.000   -6432.687   -2296.222
C(raion)[T.Бикинский район]                                      -1732.6383   1017.524     -1.703      0.089   -3729.951     264.675
C(raion)[T.Боровичский район]                                    -7000.2727   1053.653     -6.644      0.000   -9068.505   -4932.040
C(raion)[T.Буйский район]                                        -4891.1431   1053.653     -4.642      0.000   -6959.376   -2822.911
C(raion)[T.Валдайский район]                                     -6648.3136   1053.653     -6.310      0.000   -8716.546   -4580.081
C(raion)[T.Ванинский район]                                       2182.6928   1017.524      2.145      0.032     185.380    4180.006
C(raion)[T.Великий Новгород]                                     -1.514e+04   1017.524    -14.877      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Верхнебуреинский район]                               -1.513e+04   1017.524    -14.865      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Владивостокский]                                      -7449.3004   1017.524     -7.321      0.000   -9446.613   -5451.987
C(raion)[T.Волотовский район]                                      -1.5e+04   1053.564    -14.240      0.000   -1.71e+04   -1.29e+04
C(raion)[T.Вохомский район]                                      -3166.2833   1053.653     -3.005      0.003   -5234.516   -1098.051
C(raion)[T.Вяземский район]                                       4639.9189   1017.524      4.560      0.000    2642.606    6637.232
C(raion)[T.Галичский район]                                      -5046.7040   1053.653     -4.790      0.000   -7114.936   -2978.471
C(raion)[T.Город Горно-Алтайск]                                  -2596.2566   1248.146     -2.080      0.038   -5046.261    -146.252
C(raion)[T.Город Комсомо

In [45]:
difference_roads_data = difference[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_roads_data).fit()
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     2.687
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.89e-17
Time:                        23:04:24   Log-Likelihood:                -6451.7
No. Observations:                 798   AIC:                         1.319e+04
Df Residuals:                     653   BIC:                         1.387e+04
Df Model:                         144                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          127.0473    323.771      0.392      0.695    -508.711     762.805
C(raion)[T.Антроповский район]                                     207.1274    532.516      0.389      0.697    -838.523    1252.778
C(raion)[T.Анучинский район]                                       424.0304    495.479      0.856      0.392    -548.894    1396.955
C(raion)[T.Арсеньевский]                                           423.2823    433.969      0.975      0.330    -428.862    1275.426
C(raion)[T.Артемовский]                                            610.1817    433.969      1.406      0.160    -241.962    1462.326
C(raion)[T.Аяно-Майский район]                                       6.9539    433.969      0.016      0.987    -845.190     859.098
C(raion)[T.Батецкий район]                                        -151.3642    449.443     -0.337      0.736   -1033.893     731.164
C(raion)[T.Бикинский район]                                       -142.2890    433.969     -0.328      0.743    -994.433     709.855
C(raion)[T.Боровичский район]                                     -199.1946    449.443     -0.443      0.658   -1081.723     683.334
C(raion)[T.Буйский район]                                          109.1873    589.010      0.185      0.853   -1047.394    1265.769
C(raion)[T.Валдайский район]                                      -141.4744    449.443     -0.315      0.753   -1024.003     741.054
C(raion)[T.Ванинский район]                                         26.8521    433.969      0.062      0.951    -825.292     878.996
C(raion)[T.Великий Новгород]                                      1628.1248    433.969      3.752      0.000     775.981    2480.269
C(raion)[T.Верхнебуреинский район]                                1079.4391    433.969      2.487      0.013     227.295    1931.583
C(raion)[T.Владивостокский]                                        957.9168    433.969      2.207      0.028     105.773    1810.061
C(raion)[T.Волотовский район]                                       -1.0086    449.366     -0.002      0.998    -883.386     881.369
C(raion)[T.Вохомский район]                                       2376.4402    589.027      4.035      0.000    1219.825    3533.056
C(raion)[T.Вяземский район]                                        -52.6785    433.969     -0.121      0.903    -904.823     799.466
C(raion)[T.Галичский район]                                        593.6105    532.481      1.115      0.265    -451.971    1639.192
C(raion)[T.Город Горно-Алтайск]                                   1909.1990    532.515      3.585      0.000     863.550    2954.848
C(raion)[T.Город Комсомо

In [46]:
difference_housing_data = difference[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_housing_data).fit()
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.211
Method:                 Least Squares   F-statistic:                     2.505
Date:                Fri, 12 May 2023   Prob (F-statistic):           3.65e-15
Time:                        23:04:25   Log-Likelihood:                -7953.9
No. Observations:                 816   AIC:                         1.620e+04
Df Residuals:                     670   BIC:                         1.689e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1405.8276   1686.671      0.833      0.405   -1905.970    4717.625
C(raion)[T.Антроповский район]                                     724.3860   4866.948      0.149      0.882   -8831.921    1.03e+04
C(raion)[T.Анучинский район]                                      -592.7734   2608.277     -0.227      0.820   -5714.154    4528.607
C(raion)[T.Арсеньевский]                                          -158.4673   2284.652     -0.069      0.945   -4644.406    4327.472
C(raion)[T.Артемовский]                                            563.8791   2284.652      0.247      0.805   -3922.060    5049.818
C(raion)[T.Аяно-Майский район]                                    8276.4726   2284.652      3.623      0.000    3790.534    1.28e+04
C(raion)[T.Батецкий район]                                        -165.1704   2365.853     -0.070      0.944   -4810.549    4480.208
C(raion)[T.Бикинский район]                                       -240.9125   2284.652     -0.105      0.916   -4726.851    4245.026
C(raion)[T.Боровичский район]                                     -410.9709   2365.853     -0.174      0.862   -5056.350    4234.408
C(raion)[T.Буйский район]                                         -249.5459   3623.300     -0.069      0.945   -7363.936    6864.844
C(raion)[T.Валдайский район]                                      -609.4403   2469.780     -0.247      0.805   -5458.880    4240.000
C(raion)[T.Ванинский район]                                        373.6769   2284.652      0.164      0.870   -4112.262    4859.616
C(raion)[T.Великий Новгород]                                      1823.6450   2284.652      0.798      0.425   -2662.294    6309.584
C(raion)[T.Верхнебуреинский район]                                5891.3618   2284.652      2.579      0.010    1405.423    1.04e+04
C(raion)[T.Владивостокский]                                        158.7834   2284.652      0.070      0.945   -4327.155    4644.722
C(raion)[T.Волотовский район]                                     1915.6422   2365.697      0.810      0.418   -2729.430    6560.714
C(raion)[T.Вохомский район]                                       2139.3065   3100.796      0.690      0.490   -3949.140    8227.753
C(raion)[T.Вяземский район]                                        203.1508   2284.652      0.089      0.929   -4282.788    4689.090
C(raion)[T.Галичский район]                                         79.8556   2803.041      0.028      0.977   -5423.947    5583.658
C(raion)[T.Город Горно-Алтайск]                                    553.6573   2803.208      0.198      0.843   -4950.472    6057.787
C(raion)[T.Город Комсомо

In [47]:
difference_education_data = difference[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_education_data).fit()
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.617
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     7.504
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.93e-76
Time:                        23:04:26   Log-Likelihood:                -7769.0
No. Observations:                 834   AIC:                         1.583e+04
Df Residuals:                     686   BIC:                         1.653e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1117.2711   1092.377      1.023      0.307   -1027.533    3262.076
C(raion)[T.Антроповский район]                                    4996.2446   1817.864      2.748      0.006    1426.999    8565.490
C(raion)[T.Анучинский район]                                     -1476.7237   1691.451     -0.873      0.383   -4797.765    1844.318
C(raion)[T.Арсеньевский]                                         -1085.3737   1481.626     -0.733      0.464   -3994.441    1823.693
C(raion)[T.Артемовский]                                           -105.1362   1481.626     -0.071      0.943   -3014.203    2803.931
C(raion)[T.Аяно-Майский район]                                    8617.7915   1481.626      5.816      0.000    5708.725    1.15e+04
C(raion)[T.Батецкий район]                                        -129.9946   1534.265     -0.085      0.933   -3142.415    2882.425
C(raion)[T.Бикинский район]                                        492.5956   1481.626      0.332      0.740   -2416.471    3401.662
C(raion)[T.Боровичский район]                                     -442.5501   1534.265     -0.288      0.773   -3454.970    2569.870
C(raion)[T.Буйский район]                                         4418.7880   2010.750      2.198      0.028     470.824    8366.752
C(raion)[T.Валдайский район]                                      -189.5719   1534.265     -0.124      0.902   -3201.992    2822.848
C(raion)[T.Ванинский район]                                      -2133.6686   1481.626     -1.440      0.150   -5042.735     775.398
C(raion)[T.Великий Новгород]                                      7110.4670   1481.626      4.799      0.000    4201.400       1e+04
C(raion)[T.Верхнебуреинский район]                                2.198e+04   1481.626     14.835      0.000    1.91e+04    2.49e+04
C(raion)[T.Владивостокский]                                       -766.9939   1481.626     -0.518      0.605   -3676.061    2142.073
C(raion)[T.Волотовский район]                                     7615.8300   1534.182      4.964      0.000    4603.573    1.06e+04
C(raion)[T.Вохомский район]                                       2936.4299   2010.812      1.460      0.145   -1011.656    6884.516
C(raion)[T.Вяземский район]                                      -1247.9966   1481.626     -0.842      0.400   -4157.063    1661.070
C(raion)[T.Галичский район]                                       2890.3347   1817.756      1.590      0.112    -678.699    6459.368
C(raion)[T.Город Горно-Алтайск]                                   1392.4394   1817.852      0.766      0.444   -2176.781    4961.660
C(raion)[T.Город Комсомо

In [48]:
table = pd.DataFrame(np.array([actual_roads.params[(len(actual_roads.params)-10):(len(actual_roads.params))].values, 
                               actual_housing.params[(len(actual_housing.params)-10):(len(actual_housing.params))].values, 
                               actual_education.params[(len(actual_education.params)-10):(len(actual_education.params))].values,
                               initial_roads.params[(len(initial_roads.params)-10):(len(initial_roads.params))].values, 
                               initial_housing.params[(len(initial_housing.params)-10):(len(initial_housing.params))].values, 
                               initial_education.params[(len(initial_education.params)-10):(len(initial_education.params))].values,
                               difference_roads.params[(len(difference_roads.params)-10):(len(difference_roads.params))].values, 
                               difference_housing.params[(len(difference_housing.params)-10):(len(difference_housing.params))].values, 
                               difference_education.params[(len(difference_education.params)-10):(len(difference_education.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование"])
table["Тип расходов"] = pd.Series(["Фактические", "Фактические", "Фактические", "Запланированные", "Запланированные", "Запланированные", "Фактические - запланированные", "Фактические - запланированные", "Фактические - запланированные"])

In [49]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная,Тип расходов
0,-1.022505e-13,-1.658952e+03,-1185.394285,-1242.485502,-657.216793,-836.840435,-438.909412,-1281.446696,-1553.576512,-981.424852,Дорожное хозяйство,Фактические
1,-6.252589e+02,-5.590604e+02,18.462469,700.794563,603.452086,338.244438,479.553317,224.069436,1209.659261,2175.006480,ЖКХ,Фактические
2,-1.139895e+03,-3.921938e+03,-1841.033458,1678.289694,-1451.802399,-20.677160,-796.952499,1700.607891,-1279.320579,1997.812908,Образование,Фактические
3,3.850661e-14,1.608495e-13,-874.278439,-1229.716073,-1069.942389,-1243.538054,-1376.565245,-1329.822912,-829.668123,-1156.964113,Дорожное хозяйство,Запланированные
4,-2.071170e-12,2.860308e+02,239.254069,381.958006,282.340371,475.282564,-274.615461,-418.748482,151.983002,877.210556,ЖКХ,Запланированные
5,1.678496e-12,-9.953631e+02,-2517.726348,-2923.247775,-2598.876053,-1965.718080,-278.697076,-2491.323593,-2988.926811,-1550.236562,Образование,Запланированные
6,-4.595190e-14,1.493149e-13,-1340.462532,-1110.296542,-1040.960568,-753.113358,-171.650426,-1352.213958,-1621.606829,-812.079171,Дорожное хозяйство,Фактические - запланированные
7,3.952477e-13,-5.766960e+02,414.968909,403.832712,652.695796,168.266844,570.984006,233.631776,687.757640,1611.809169,ЖКХ,Фактические - запланированные
8,-9.563509e-13,-1.224622e+03,-311.230828,4570.157101,1242.148885,2253.545650,-1636.537420,4076.337177,888.891357,3245.551653,Образование,Фактические - запланированные


In [50]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_fin_75_cor.xlsx")